In [ ]:
from header import *
from scipy import optimize
plt.rcParams.update({'font.size': 16})
plots_dir = Path('imgs/')
timelimit = 600
memorylimit = 10000

In [ ]:
def plot_scaling(df, x, y, prefix, hline=False, algo1=None, algo2=None, show_mean=False, trend_line='', xlog=False, ylog=False, title=None, fit_only=None):
    #fig, ax = plt.subplots()
    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(6, 4, forward=True)
    
    # grid
    ax.set_facecolor('#F3F3F3')
    
    # ticks
    ax.tick_params(
        axis='both',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=True,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
    #    labelbottom=False # labels along the bottom edge are off
        labelsize=16,
    )
    
    #ax.tick_params(
    #    axis='y',          # changes apply to the x-axis
    #    which='minor',      # both major and minor ticks are affected
    #    left=False,         # ticks along the top edge are off
    #    labelbottom=False # labels along the bottom edge are off
    #)
    
    for spine in ["top", "left", "right"]:
        ax.spines[spine].set_visible(False)
    
    # Data
    d = df.groupby(['alg', x], group_keys=False, observed=True)[y].mean()  # observed=True drops categorical algos that don't appear
    d = d.reset_index(name='mean_{}'.format(y))
    d = d.pivot(index=x, columns='alg', values='mean_{}'.format(y))
   
    for algo in d.columns:
        d[algo] = d[algo].dropna()[d[algo] > 0]
        #s = s[s>0]
        
    #plot_fit(d, ax, )

    if trend_line == "exp":        
        z = {}
        for algo in d.columns:
            s = d[algo].dropna()
            z[algo] = np.polyfit(s.index, np.log(s), 1)
        xs = list(d.index)
        if show_mean:
            d = d.append(df.groupby('alg')[y].mean().rename('all'))
        #xs.append(xs[-1]*1.1)  # extrapolation
        
        # ln(y[i]) ~= a*x[i] + b
        #  => y[i] ~= exp(a*x[i] + b)
        # y[i] = O(base^i) = C * base^i
        #  => a*x[i] + b = ln(C * base^x[i]) = ln(base)*x[i] + ln(C)
        #  => a*x[0] + b = ln(base)*x[0] + ln(C)
        #     a*x[1] + b = ln(base)*x[1] + ln(C)
        #  => a*(x[1]-x[0]) = ln(base)*(x[1]-x[0]) => base = exp(a)
        
        # Best fit lines
        for algo in d.columns:
            regression_line = []
            a, b = z[algo]
            #angle = math.atan2(a, abs(b)) * 180 / math.pi
            for i in xs:
                regression_line.append(np.exp(a*i+b))
            #display(regression_line)
            ax.plot(xs, regression_line, linestyle='-', color=algo2color(algo), alpha=0.8)
            weight = 'bold' if 'seeds' in algo else 'normal'
            label = '$\sim {{{:0.2f}}}^x$'.format(np.exp(a*(s.index[1]-s.index[0])))
            ax.text(xs[-1], regression_line[-1], algo2beautiful(algo) + label,
                    color=algo2color(algo), ha='center', va='bottom', size=15, alpha=1, weight=weight)
    elif trend_line == "poly":
        # ln(y) = aln(x)+b
        # y = e^(aln(x)+b)
        # y = e^b * x^a
        
        z = {}
        for algo in d.columns:
            s = d[algo].dropna()
            #s = s[s>0]
            z[algo] = np.polyfit(np.log(s.index), np.log(s), 1)
        xs = list(d.index)
        if show_mean:
            d = d.append(df.groupby('alg')[y].mean().rename('all'))
        #xs.append(xs[-1]*1.1)  # extrapolation
        
        # log(y[i]) ~= a*log(x[i]) + b
        #  => y[i] ~= e^(a*log(x[i]) + b) = x[i]^a * e^b
        # y[i] = O(base^i) = C * base^i
        #  => a*x[i] + b = ln(C * base^x[i]) = ln(base)*x[i] + ln(C)
        #  => a*x[0] + b = ln(base)*x[0] + ln(C)
        #     x[i]*[ * + b = ln(base)*x[1] + ln(C)
        #  => a*(x[1]-x[0]) = ln(base)*(x[1]-x[0]) => base = exp(a)
        
        # Best fit lines
        for algo in d.columns:
            regression_line = []
            a, b = z[algo]
            #angle = math.atan2(a, abs(b)) * 180 / math.pi
            for i in xs:
                regression_line.append((i**a) * np.exp(b))
            #display(regression_line)
            
            weight = 'bold' if 'seeds' in algo else 'normal'
            label = ''
            if len(d[algo].dropna()) > 1:
                ax.plot(xs, regression_line, linestyle='-', color=algo2color(algo), alpha=0.8)
                #print(np.exp(b), a)
                label = '$\sim x^{{{:0.2f}}}$'.format(a)  ## np.exp(b)*x^a
            ax.text(xs[-1], regression_line[-1], algo2beautiful(algo) + label,
                    color=algo2color(algo), ha='center', va='bottom', size=15, alpha=1, weight=weight)
    elif trend_line == "lin_poly":
        z = {}
        f = lambda x, a, b, c: a*x + b*x**c
        fe = lambda x, a, b, c: np.log(a*x + b*x**c)
        for algo in d.columns:
            s = d[algo].dropna()
            params = optimize.curve_fit(fe, 
                                         s.index, 
                                         np.log(s), 
                                         p0=[1e-10, 1e-10, 2.5],
                                         bounds=([1e-20,1e-20,1], [1e-5, 1e-5, 3])
                                        )[0]
            if algo=='pa':
                z[algo+'_lin'] = [params[0], 0, 0]
                z[algo+'_poly'] = [0, params[1], params[2]]
            else:
                z[algo] = params
            #a, b, c = params
            #xx = s.index.min()
            #print('for xmin: ', a*xx / (b*xx**c))
            #xx = s.index.max()
            #print('for xmax', a*xx / (b*xx**c))
        xs = list(d.index)
        #if show_mean:
        #    d = d.append(df.groupby('alg')[y].mean().rename('all'))
        
        # Best fit lines
        j = 0
        for algo in z:
            j += 1
            regression_line = []
            params = z[algo]
            #print(algo, params)
            #angle = math.atan2(a, abs(b)) * 180 / math.pi
            for i in xs:
                regression_line.append(f(i, *params))
            #display(regression_line)
            
            weight = algo == 'pa'
            label = ''
            #if len(d[algo].dropna()) > 1:
            ax.plot(xs, regression_line, linestyle='-', color=algo2color(algo), alpha=0.8)
            #label = '$\sim {:0.2f} x + {:0.2f} * x^{{{:0.2f}}}$'.format(*params)
            label = '$\sim x^{{{:0.2f}}}$'.format(params[2])
            
            ax.text(xs[-1], regression_line[-1], algo2beautiful(algo) + label,
                    color=algo2color(algo), ha='center', va='bottom', size=15, alpha=1, weight=weight)
    elif trend_line == "nlogn":
        z = {}
        f = lambda x, a, b: a*x + b*x * np.log(x)
        fe = lambda x, a, b: np.log(a*x +b*x* np.log(x))
        for algo in d.columns:
            s = d[algo].dropna()
            z[algo] = optimize.curve_fit(fe, 
                                         s.index, 
                                         np.log(s), 
                                         p0=[1e-10, 1e-10],
                                         bounds=([1e-20, 1e-20], [1e-5, 1e-5])
                                        )[0]
        xs = list(d.index)
        if show_mean:
            d = d.append(df.groupby('alg')[y].mean().rename('all'))
        
        # Best fit lines
        j = 0
        for algo in d.columns:
            j += 1
            regression_line = []
            params = z[algo]
            #print(algo, params)
            #angle = math.atan2(a, abs(b)) * 180 / math.pi
            for i in xs:
                regression_line.append(f(i, *params))
            #display(regression_line)
            
            weight = algo == 'pa'
            label = ''
            if len(d[algo].dropna()) > 1:
                ax.plot(xs, regression_line, linestyle='-', color=algo2color(algo), alpha=0.8)
                label = '$\sim {:0.2f} * x + {:0.2f} * x lg (x)$'.format(*params)
            ax.text(xs[-1], regression_line[-1], algo2beautiful(algo) + label,
                    color=algo2color(algo), ha='center', va='bottom', size=15, alpha=1, weight=weight)
    elif trend_line:
        print(trend_line)
        assert(False)
    
    # ticks
    if x == 'error_rate':
        ax.xaxis.set_major_formatter(ticker.PercentFormatter(xmax=1.0, decimals=0))
    
    if x == 'cost':
        ax.set_xticks(np.arange(0, max(df[x])+1, 1))
    
    #arr = [ 0 ] + [ 10**n for n in range(0,int(math.log10(max(df[y])))+1) ]
    #ax.set_yticks(arr)

    # Times
    #for index, row in d.iterrows():
        #print(row["astar-seeds"])
        #print(row["astar-seeds-intervals"])
    #    label = '{0:.3g}x'.format(row[algo1] / row[algo2])
    #    tmp_x = index if index != 'all' else len(d)-1
    #    tmp_y = np.exp((np.log(row[0]) + np.log(row[1])) / 2)
    #    ax.text(tmp_x, tmp_y, label, ha='right', va='center', size=12, alpha=0.7)
        
    colors = [ algo2color(algo) for algo in d.columns ]
    d.columns = [ algo2beautiful(col) for col in d.columns ]
    
    ########### Bars
    d.plot(ax=ax, alpha=0.6, zorder=3, rot=0, color=colors, marker='o', ls='', legend=False)
    #d.plot.bar(ax=ax, alpha=0.5, width=0.7, zorder=3, rot=0, color=colors)
    
    #if hline:
    #    ax.legend(loc="upper left", bbox_to_anchor=(0.0, 0.95), prop={'size': 15}, framealpha=0.5)
    #else:
    #    ax.legend(loc="upper left", prop={'size': 15}, framealpha=0.5)

    if hline:
        if y == 'explored_states' or y == 'crumbs':
            miny = df.len.median()
            maxy = df.refsize.median()*df.len.median()
        elif y == 'explored_per_bp':
            miny = 1
            maxy = df.refsize.median()
        else:
            assert(False)
        
        ax.axhline(maxy, color='k', linestyle='--', label=df.ref[0]) #, xmin=min(df[x]), xmax=max(df[x])+1
        ax.text(0.0, maxy, 'max    ', ha='right', va='center', size=16)
        
        ax.axhline(y=miny, color='k', linestyle='--', label=df.ref[0])  # xmin=min(df[x]), xmax=max(df[x])+1, 
        ax.text(0.0, miny, 'min    ', ha='right', va='center', size=16)

    if ylog:
        ax.set_yscale('log')
    else:
        ax.set_ylim(0)
        
    if xlog:
        ax.set_xscale('log')
      
       
    #ax.grid(True, which='both', axis='y', color='w')
    #ax.grid(False, axis='x', which='both')
    ax.grid(True, axis='y', which='major', color='w')
    ax.grid(False, axis='y', which='minor')
    
    # axis labelsi
    ax.set_xlabel(col2name(x), size=18)  # weight='bold',
    ax.set_ylabel(col2name(y), rotation=0, ha='left', size=18)
    ax.yaxis.set_label_coords(-0.10,1.00)
    
    if y == 't(map)':
        ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda num, pos: myticks(num, pos)+'s'))
   
    filename = '{}_{}_{}'.format(prefix, x, y)
    
    if trend_line:
        title = '{}_{}'.format(trend_line, title)
    
    if title:
        ax.set_title(title)
        filename += '_{}'.format(title)
    
    #plt.savefig(prefix+'_'+x+'-'+y+'.pdf', bbox_inches='tight')
    plt.savefig(plots_dir/(filename+'.pdf'), bbox_inches='tight')

In [ ]:
# TOOL COMPARISON -- uses best params for PA
experiment_name = "length_scaling"
df = read_benchmarks('table/tools_N1e7.tsv')

#display(df[ (df.alg == 'wfa') & (df.e == 0.01) ])

df = df[ df.s < timelimit ]
df = df[ df.max_vms < memorylimit ]
#df = df[ (df.alg == 'wfa') & (df.e == 0.01) ]
df = df[df.n >= 10**4]
df = df[df.alg != 'dijkstra']
#df = df[df.alg != "wfa"]

#display(df.head())

def f(row):
    return row.n >= 10000

for e in pd.unique(df.e):
    df_n = df[df.e == e]
    #df_n = df_n.groupby(["alg", "cnt", "e"]).median()
    #display(df_n)
    display(df_n[df_n.alg == "pa"])
    plot_scaling(df_n, y='s_per_pair', x='n', prefix=experiment_name+'e={}'.format(e), xlog=True, ylog=True, trend_line='poly', title='e={}'.format(e), fit_only=f)
    #plot_scaling(df_n, y='s_per_pair', x='n', prefix=experiment_name, xlog=True, ylog=True, trend_line='poly', title='e={}'.format(e))
    #plot_scaling(df_e, y='s_per_pair', x='e', prefix=experiment_name, xlog=True, ylog=True)

In [ ]:
experiment_name = 'linear'

df = read_benchmarks('table/params_N1e7.tsv')

plot_scaling(df, y='s_per_pair', x='n', prefix=experiment_name, xlog=True, ylog=True, trend_line="poly")
plot_scaling(df, y='s_per_pair', x='n', prefix=experiment_name, xlog=True, ylog=True, trend_line="nlogn")
plot_scaling(df, y='precom', x='n', prefix=experiment_name, xlog=True, ylog=True, trend_line="poly")
plot_scaling(df, y='align', x='n', prefix=experiment_name, xlog=True, ylog=True, trend_line="poly")
plot_scaling(df, y='align_frac', x='n', prefix=experiment_name, xlog=True, ylog=False)
plot_scaling(df, y='prune_frac', x='n', prefix=experiment_name, xlog=True, ylog=False)
plot_scaling(df, y='h_approx_frac', x='n', prefix=experiment_name, xlog=True, ylog=False)
plot_scaling(df, y='expanded_frac', x='n', prefix=experiment_name, xlog=True, ylog=False)

In [ ]:
experiment_name = 'params'

df = read_benchmarks('table/params_N1e7.tsv')

for n in pd.unique(df.n):
    iфf str(n)[0] == '3' or len(str(n)) % 2:
        continue
    for e in [0.2]: # pd.unique(df.e):
        df_sub = df[(df.n == n) & (df.e == e)]
        display(df_sub)
        plot_scaling(df_sub, y='band', x='k', prefix=experiment_name, xlog=True, ylog=False, title='n={}, e={}'.format(n,e))

In [ ]:
df = read_benchmarks('table/tools_N100000000.tsv')
df = df[df.e == 0.1] 
df = df[df.alg == 'pa']
display(df)

In [ ]:
# SCALING WITH e
experiment_name = "e_scaling"
df = read_benchmarks('table/tools_N1e7.tsv')
df = df[df.n == 10000]
#display(df)
plot_scaling(df, y='s_per_pair', x='e', prefix=experiment_name, xlog=False, ylog=True, trend_line=False)

In [ ]:
# Best params per input
df = read_benchmarks('table/params_N1e7.tsv')

# Data
d = df.loc[df.groupby(['alg', 'cnt', 'n', 'e']).s.idxmin()]
display(d[d.e == 0.1])

In [ ]:
# Parameter comparison: k
# e=0.01: k=20..30, m=0
# e=0.05: k=10..15, m=0
# e=0.20: k=8..10, m=1, for large n, k>=10
experiment_name = 'params_k'

df = read_benchmarks('table/params_N1e7.tsv')

for n in pd.unique(df.n):
    df2 = df[(df.e == 0.05) & (df.pf == 1) & (df.n == n)]
    #display(df2)
    plot_scaling(df2, y='s_per_pair', x='k', prefix=experiment_name, xlog=False, ylog=False, title=f'n = {n}')

In [ ]:
# Parameter comparison: pf
# pf doesn't matter much => fix to 1.
experiment_name = 'params_pf'

df = read_benchmarks('table/params_N1000000.tsv')

for n in pd.unique(df.n):
    df2 = df[(df.e == 0.05) & (df.n == n) & (df.k == 15)]
    #display(df2)
    plot_scaling(df2, y='s_per_pair', x='pf', prefix=experiment_name, xlog=False, ylog=False, title=f'n = {n}')